In [4]:
import os
import uproot
import numpy as np
import pandas as pd

# Configuración
sim_dir = r"C:\Users\conej\Documents\Universidad\Geant4\Projects\GIT-3\build\Release"
num_simulations = 17  # Sim0.root hasta Sim16.root (17 archivos)
thickness = 6.3  # Thickness en cm
energies = np.arange(20, 105, 5)  # Energías de 20 keV a 100 keV

# Función para procesar el archivo ROOT y obtener hits y energía promedio
def process_root_file(root_filename):
    file_path = os.path.join(sim_dir, root_filename)
    if not os.path.isfile(file_path):
        return -1, -1  # Si el archivo no existe, devolver valores inválidos
    
    try:
        with uproot.open(file_path) as file:
            tree = file["G4_PCM"]  # Asegúrate que el nombre del TTree es correcto
            df = tree.arrays(library="pd")  # Convertir a DataFrame
        
        # Filtrar las energías mayores que 0
        df_filtered = df[df['Energy'] > 0]
        hits = len(df_filtered)
        if hits == 0:
            return 0, 0
        
        # Calcular el promedio de las energías
        avener = df_filtered['Energy'].mean()
        return hits, avener
    except Exception as e:
        print(f"Error procesando {root_filename}: {e}")
        return -1, -1

# Función para calcular el coeficiente de atenuación
def calculate_coeff(thickness, avener, energy, hits):
    # Thickness ya está en cm
    coeff = (-1 / thickness) * np.log((avener / energy) * (hits / 1000000)) / 1.036031746031746
    return coeff

# Procesar todos los archivos ROOT y calcular coeficientes
data = []
for i in range(num_simulations):
    root_filename = f"Sim{i}.root"
    energy = energies[i]
    hits, avener = process_root_file(root_filename)
    
    if hits > 0 and avener > 0:
        coeff = calculate_coeff(thickness, avener, energy, hits)
        data.append([energy, coeff])
    else:
        print(f"Archivo {root_filename} no pudo ser procesado correctamente.")

# Convertir los resultados a un DataFrame
df_results = pd.DataFrame(data, columns=["Energy (keV)", "Attenuation Coeff"])

# Guardar los resultados en un archivo CSV
output_csv = os.path.join(sim_dir, "attenuation_coefficients.csv")
df_results.to_csv(output_csv, index=False)

print(f"Archivo CSV guardado en: {output_csv}")


Archivo CSV guardado en: C:\Users\conej\Documents\Universidad\Geant4\Projects\GIT-3\build\Release\attenuation_coefficients.csv


In [2]:
thick = 6.3
TS = 0.15*2
TM = 2.5*2
TA = 0.5*2
skinD = 1.09
muscleD = 1.05
adiposeD = 0.95

In [3]:
(skinD*(TS/thick))+(muscleD*(TM/thick))+(adiposeD*(TA/thick))

1.036031746031746